In [1]:
!pip install pandas pyarrow fastparquet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.8/683.8 kB 9.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [fastparquet]


In [20]:
import pandas as pd
import json

In [3]:
df = pd.read_parquet("data.parquet")


In [4]:
df.head()       # Show first 5 rows
df.info()       # Summary of columns and data types
df.describe()   # Statistical summary of numeric columns
df.shape        # Number of rows and columns
df.columns      # Column names


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3000 non-null   object 
 1   base_id          3000 non-null   object 
 2   sources          3000 non-null   object 
 3   names            3000 non-null   object 
 4   categories       2957 non-null   object 
 5   confidence       3000 non-null   float64
 6   websites         2459 non-null   object 
 7   socials          2090 non-null   object 
 8   emails           0 non-null      object 
 9   phones           2765 non-null   object 
 10  brand            884 non-null    object 
 11  addresses        3000 non-null   object 
 12  base_sources     3000 non-null   object 
 13  base_names       3000 non-null   object 
 14  base_categories  2960 non-null   object 
 15  base_confidence  3000 non-null   float64
 16  base_websites    2912 non-null   object 
 17  base_socials  

Index(['id', 'base_id', 'sources', 'names', 'categories', 'confidence',
       'websites', 'socials', 'emails', 'phones', 'brand', 'addresses',
       'base_sources', 'base_names', 'base_categories', 'base_confidence',
       'base_websites', 'base_socials', 'base_emails', 'base_phones',
       'base_brand', 'base_addresses'],
      dtype='object')

In [7]:
df["addresses"].head(5) #see what kind of data is in the addesses column

0    [{"freeform":"Av. Carlos Cano, Glorieta La Mon...
1    [{"freeform":"609 W Maple Ave","locality":"Spr...
2    [{"freeform":"445 W 45th St","locality":"New Y...
3    [{"freeform":"401 E Chestnut St Unit 510","loc...
4    [{"freeform":"Rua Capitão Inácio Prata, S/N","...
Name: base_addresses, dtype: object

In [8]:
df[["addresses", "base_addresses"]].head(10)

,addresses,base_addresses
0,"[{""freeform"":""Avenida Carlos Cano"",""locality"":...","[{""freeform"":""Av. Carlos Cano, Glorieta La Mon..."
1,"[{""freeform"":""609 W Maple Ave"",""locality"":""Spr...","[{""freeform"":""609 W Maple Ave"",""locality"":""Spr..."
2,"[{""freeform"":""688 9th Ave"",""locality"":""New Yor...","[{""freeform"":""445 W 45th St"",""locality"":""New Y..."
3,"[{""freeform"":""401 E Chestnut St Unit 790"",""loc...","[{""freeform"":""401 E Chestnut St Unit 510"",""loc..."
4,"[{""freeform"":""Avenida Capitão Inácio Prata, Sn...","[{""freeform"":""Rua Capitão Inácio Prata, S/N"",""..."
5,"[{""freeform"":""825 E 1180 S"",""locality"":""Americ...","[{""freeform"":""825 E 1180 S Ste 300"",""locality""..."
6,"[{""country"":""TR""}]","[{""country"":""TR""}]"
7,"[{""freeform"":""Calle Milicias, 4"",""locality"":""L...","[{""freeform"":""Parque del Carmen, 6"",""locality""..."
8,"[{""freeform"":""Alliance hospital, Vijay Colony ...","[{""freeform"":""Alliance Hospital, Vijay Colony ..."
9,"[{""freeform"":""1100 W Stewart Dr"",""locality"":""O...","[{""freeform"":""1000 W La Veta Dr, 1st Floor"",""l..."


In [22]:
from IPython.display import display

for i, row in df[["addresses", "base_addresses"]].head(5).iterrows():
    print(f"🔹 Row {i}")
    print("addresses:      ", row['addresses'])
    print("base_addresses: ", row['base_addresses'])
    print("-" * 80)


🔹 Row 0
addresses:       [{"freeform":"Avenida Carlos Cano","locality":"Los Barrios","postcode":"11370","country":"ES"}]
base_addresses:  [{"freeform":"Av. Carlos Cano, Glorieta La Montera","locality":"Los Barrios","region":"Cádiz","country":"ES","postcode":"11370"}]
--------------------------------------------------------------------------------
🔹 Row 1
addresses:       [{"freeform":"609 W Maple Ave","locality":"Springdale","postcode":"72764-5335","region":"AR","country":"US"}]
base_addresses:  [{"freeform":"609 W Maple Ave","locality":"Springdale","region":"AR","country":"US","postcode":"72764"}]
--------------------------------------------------------------------------------
🔹 Row 2
addresses:       [{"freeform":"688 9th Ave","locality":"New York","postcode":"10036-3657","region":"NY","country":"US"}]
base_addresses:  [{"freeform":"445 W 45th St","locality":"New York","postcode":"10036-3502","region":"NY","country":"US"}]
-------------------------------------------------------------

In [25]:
def safe_parse(x):
    try:
        return json.loads(x) if isinstance(x, str) else x
    except Exception:
        return None

df["addresses_parsed"] = df["addresses"].apply(safe_parse)
df["base_addresses_parsed"] = df["base_addresses"].apply(safe_parse)

def normalize_json(obj):
    if isinstance(obj, list):
        obj = obj[0] if len(obj) > 0 else {}
    if isinstance(obj, dict):
        return json.dumps(obj, sort_keys=True)
    return str(obj)

df["addr_norm"] = df["addresses_parsed"].apply(normalize_json)
df["base_addr_norm"] = df["base_addresses_parsed"].apply(normalize_json)

df_unique = df[df["addr_norm"] != df["base_addr_norm"]].copy() # drop rows that have the same values 

print(f"Original rows: {len(df)}")
print(f"Remaining rows after dropping matches: {len(df_unique)}")

Original rows: 3000
Remaining rows after dropping matches: 2466


In [21]:
for i, row in df_unique[["addresses", "base_addresses"]].head(10).iterrows():
    print(f"🔹 Row {i}")
    print("addresses:      ", row['addresses'])
    print("base_addresses: ", row['base_addresses'])
    print("-" * 80)


🔹 Row 0
addresses:       [{"freeform":"Avenida Carlos Cano","locality":"Los Barrios","postcode":"11370","country":"ES"}]
base_addresses:  [{"freeform":"Av. Carlos Cano, Glorieta La Montera","locality":"Los Barrios","region":"Cádiz","country":"ES","postcode":"11370"}]
--------------------------------------------------------------------------------
🔹 Row 1
addresses:       [{"freeform":"609 W Maple Ave","locality":"Springdale","postcode":"72764-5335","region":"AR","country":"US"}]
base_addresses:  [{"freeform":"609 W Maple Ave","locality":"Springdale","region":"AR","country":"US","postcode":"72764"}]
--------------------------------------------------------------------------------
🔹 Row 2
addresses:       [{"freeform":"688 9th Ave","locality":"New York","postcode":"10036-3657","region":"NY","country":"US"}]
base_addresses:  [{"freeform":"445 W 45th St","locality":"New York","postcode":"10036-3502","region":"NY","country":"US"}]
-------------------------------------------------------------